In [8]:

import gurobipy as gp
from gurobipy import *
import numpy as np
import math
import copy
import pandas as pd
import matplotlib.pyplot as plt

# Model
file_path = "data_small.txt"
with open(file_path, "r") as f:  # Read the data file
    data = f.readlines()

print(data)

VRP = []                                    # Create array for data related to nodes
i=0                                         # to keep track of lines in data file
for line in data:
    i=i+1
    words = line.split()
    words=[int(i) for i in words]           # Covert data from string to integer
    VRP.append(words)                       # Store node data
VRP = np.array(VRP)

print(VRP)




['0\t40\t50\t0\t0\t1236\t0\n', '1\t40\t69\t20\t482\t842\t90\n', '2\t40\t66\t20\t18\t378\t90\n', '3\t38\t68\t20\t110\t470\t90\n', '4\t38\t70\t10\t390\t750\t90\n', '5\t25\t85\t20\t507\t867\t90\n', '6\t22\t75\t30\t30\t390\t90\n', '7\t22\t85\t10\t414\t774\t90\n', '8\t25\t50\t10\t15\t375\t90\n', '9\t25\t52\t40\t17\t377\t90\n', '10\t25\t55\t10\t482\t842\t90\n', '11\t23\t52\t10\t109\t469\t90\n', '12\t23\t55\t20\t390\t750\t90\n', '13\t26\t32\t10\t668\t1028\t90\n', '14\t25\t30\t10\t576\t936\t90\n', '15\t25\t35\t10\t761\t1121\t90\n', '16\t40\t5\t30\t231\t591\t90\n', '17\t40\t15\t40\t35\t395\t90\n', '18\t38\t5\t30\t323\t683\t90\n', '19\t38\t15\t10\t516\t876\t90']
[[   0   40   50    0    0 1236    0]
 [   1   40   69   20  482  842   90]
 [   2   40   66   20   18  378   90]
 [   3   38   68   20  110  470   90]
 [   4   38   70   10  390  750   90]
 [   5   25   85   20  507  867   90]
 [   6   22   75   30   30  390   90]
 [   7   22   85   10  414  774   90]
 [   8   25   50   10   15  375   9